In [1]:
import findspark
import os
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f

os.environ["SPARK_HOME"] = "/home/nphung/spark-3.3.1-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

findspark.init()

spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.driver.memory", "2G")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .getOrCreate()
)

print("Spark session is now available")

your 131072x1 screen size is bogus. expect trouble


23/01/26 20:08:55 WARN Utils: Your hostname, DESKTOP-9EFE6HH resolves to a loopback address: 127.0.1.1; using 172.24.61.140 instead (on interface eth0)
23/01/26 20:08:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/26 20:08:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/26 20:08:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/26 20:08:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
Spark session is now available


In [2]:
# def read_csv(path: str)->DataFrame:
#     return spark.read.csv(path=path, header=True)

In [3]:
from pyspark_template.readers.csv import read_csv

clinical_trials: DataFrame = read_csv(spark, "../data/clinical_trials.csv")
clinical_trials.show(20, truncate=False)

+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------------------------------+
|id         |scientific_title                                                                                                                                                     |date          |journal                             |
+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+------------------------------------+
|NCT01967433|Use of Diphenhydramine as an Adjunctive Sedative nphung for Colonoscopy in Patients Chronically on Opioids                                                           |1 January 2020|Journal of emergency nursing        |
|NCT04189588|Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Inject

In [4]:
drugs: DataFrame = read_csv(spark, "../data/drugs.csv")
drugs

atccode,drug
A04AD,DIPHENHYDRAMINE
S03AA,TETRACYCLINE
V03AB,ETHANOL
A03BA,ATROPINE
A01AD,EPINEPHRINE
6302001,ISOPRENALINE
R01AD,BETAMETHASONE
HELLO,CORONA


In [5]:
pubmed: DataFrame = read_csv(spark, "../data/pubmed.csv")
pubmed.show(20, truncate=True)

+---+--------------------+----------+--------------------+
| id|               title|      date|             journal|
+---+--------------------+----------+--------------------+
|  1|A 65-year-old man...|01/01/2019|Journal of emerge...|
|  2|An evaluation of ...|01/01/2019|Journal of emerge...|
|  3|Diphenhydramine h...|02/01/2019|The Journal of pe...|
|  4|Tetracycline Resi...|01/01/2020|Journal of food p...|
|  5|Appositional Tetr...|02/01/2020|American journal ...|
|  6|Rapid reacquisiti...|2020-01-01|  Psychopharmacology|
|  7|The High Cost of ...|01/02/2020|The journal of al...|
|  8|Time to epinephri...|01/03/2020|The journal of al...|
+---+--------------------+----------+--------------------+



In [6]:
# JSON is broken
# pubmed_json: DataFrame = spark.read.json("../data/pubmed.json")
# pubmed_json

In [7]:
from pyspark_template.transform.common import prefix_cols

pubmed_final = prefix_cols(pubmed, "pubmed")
clinical_trials_final = prefix_cols(clinical_trials, "clinical_trials")

In [8]:
pubmed_final

pubmed_id,pubmed_title,pubmed_date,pubmed_journal
1,A 65-year-old man...,01/01/2019,Journal of emerge...
2,An evaluation of ...,01/01/2019,Journal of emerge...
3,Diphenhydramine h...,02/01/2019,The Journal of pe...
4,Tetracycline Resi...,01/01/2020,Journal of food p...
5,Appositional Tetr...,02/01/2020,American journal ...
6,Rapid reacquisiti...,2020-01-01,Psychopharmacology
7,The High Cost of ...,01/02/2020,The journal of al...
8,Time to epinephri...,01/03/2020,The journal of al...


In [9]:
clinical_trials_final

clinical_trials_id,clinical_trials_scientific_title,clinical_trials_date,clinical_trials_journal
NCT01967433,Use of Diphenhydr...,1 January 2020,Journal of emerge...
NCT04189588,Phase 2 Study IV ...,1 January 2020,Journal of emerge...
NCT04237090,,1 January 2020,Journal of emerge...
NCT04237091,Feasibility of a ...,1 January 2020,Journal of emerge...
NCT04153396,Preemptive 1234 W...,1 January 2020,Hôpitaux Universi...
NCT03490942,Glucagon Infusion...,25/05/2020,null
null,Glucagon Infusion...,25/05/2020,Journal of emerge...
NCT04188184,Tranexamic Acid n...,27 April 2020,Journal of emerge...
NCT04188185,Corana nphung Epi...,27 April 2020,Journal of emerge...


In [10]:
# Un drug est considéré comme mentionné dans un article PubMed ou un essai clinique s’il est mentionné dans le titre de la publication.
# Un drug est considéré comme mentionné par un journal s’il est mentionné dans une publication émise par ce journal.

from quinn.extensions import isTrue

from pyspark_template.writers.json import write_json

result: DataFrame = (
    drugs.crossJoin(pubmed_final)
    .crossJoin(clinical_trials_final)
    .withColumn(
        "pubmed_title_title_array", f.split(f.upper(f.col("pubmed_title")), " ")
    )
    .withColumn(
        "clinical_trials_scientific_title_array",
        f.split(f.upper(f.col("clinical_trials_scientific_title")), " "),
    )
    .withColumn(
        "pubmed_title_title_array",
        f.transform(
            f.col("pubmed_title_title_array"), lambda x: f.regexp_replace(x, ",", "")
        ),
    )
    .withColumn(
        "title?",
        (
            (
                f.array_contains(
                    f.col("pubmed_title_title_array"), f.upper(f.col("drug"))
                )
            )
            | (
                f.array_contains(
                    f.col("clinical_trials_scientific_title_array"),
                    f.upper(f.col("drug")),
                )
            )
        ),
    )
    .drop("pubmed_title_title_array", "clinical_trials_scientific_title_array")
    .where(f.col("title?").isTrue())
    .groupby("atccode", "drug")
    .agg(
        f.collect_set(
            f.struct(
                f.col("pubmed_id"),
                f.col("pubmed_title"),
                f.col("pubmed_date"),
                f.col("pubmed_journal"),
            )
        ).alias("pubmeds"),
        f.collect_set(
            f.struct(
                f.col("clinical_trials_id"),
                f.col("clinical_trials_scientific_title"),
                f.col("clinical_trials_date"),
                f.col("clinical_trials_journal"),
            )
        ).alias("clinical_trials"),
        f.collect_set(
            f.struct(
                f.col("pubmed_date"),
                f.col("pubmed_journal"),
                f.col("clinical_trials_date"),
                f.col("clinical_trials_journal"),
            )
        ).alias("journals"),
    )
)
result

# final = result

# Sortie un JSON drug <->
# write_json(final, "final.json", "final_single.json")

atccode,drug,pubmeds,clinical_trials,journals
A01AD,EPINEPHRINE,"[{6, Rapid reacqu...","[{NCT04188185, Co...","[{01/01/2019, Jou..."
S03AA,TETRACYCLINE,"[{6, Rapid reacqu...","[{NCT04188185, Co...","[{02/01/2020, Ame..."
R01AD,BETAMETHASONE,"[{6, Rapid reacqu...","[{NCT04153396, Pr...","[{2020-01-01, Psy..."
V03AB,ETHANOL,"[{6, Rapid reacqu...","[{NCT04188185, Co...","[{2020-01-01, Psy..."
A04AD,DIPHENHYDRAMINE,"[{6, Rapid reacqu...","[{NCT04188185, Co...","[{01/01/2019, Jou..."


In [11]:
from pyspark_template.transform.common import prepared_title_array

prepared_pubmed_final: DataFrame = prepared_title_array(pubmed_final, "pubmed_title")

drugs_pubmed: DataFrame = (
    drugs.join(
        prepared_pubmed_final,
        f.array_contains(f.col("pubmed_title_array"), f.upper(f.col("drug"))),
        "outer",
    )
    .drop("pubmed_title_array")
    .groupby("atccode", "drug")
    .agg(
        f.collect_set(
            f.struct(
                f.col("pubmed_id"),
                f.col("pubmed_title"),
                f.col("pubmed_date"),
                f.col("pubmed_journal"),
            )
        ).alias("pubmeds")
    )
)

drugs_pubmed

# write_json(drugs_pubmed, "drugs_pubmed.json", "drugs_pubmed_single.json")

atccode,drug,pubmeds
S03AA,TETRACYCLINE,"[{6, Rapid reacqu..."
A01AD,EPINEPHRINE,"[{8, Time to epin..."
V03AB,ETHANOL,"[{6, Rapid reacqu..."
A04AD,DIPHENHYDRAMINE,"[{3, Diphenhydram..."
A03BA,ATROPINE,"[{null, null, nul..."
6302001,ISOPRENALINE,"[{null, null, nul..."
R01AD,BETAMETHASONE,"[{null, null, nul..."
HELLO,CORONA,"[{null, null, nul..."


In [12]:
prepared_clinical_trials_final: DataFrame = prepared_title_array(
    clinical_trials_final, "clinical_trials_scientific_title"
)

drugs_pubmed_clinical_trials: DataFrame = (
    drugs_pubmed.join(
        prepared_clinical_trials_final,
        f.array_contains(
            f.col("clinical_trials_scientific_title_array"), f.upper(f.col("drug"))
        ),
        "left",
    )
    .groupby("atccode", "drug", "pubmeds")
    .agg(
        f.collect_set(
            f.struct(
                f.col("clinical_trials_id"),
                f.col("clinical_trials_scientific_title"),
                f.col("clinical_trials_date"),
                f.col("clinical_trials_journal"),
            )
        ).alias("clinical_trials")
    )
)

drugs_pubmed_clinical_trials

# write_json(drugs_pubmed_clinical_trials, "drugs_pubmed_clinical_trials.json", "drugs_pubmed_clinical_trials_single.json")

atccode,drug,pubmeds,clinical_trials
S03AA,TETRACYCLINE,"[{6, Rapid reacqu...","[{null, null, nul..."
A01AD,EPINEPHRINE,"[{8, Time to epin...","[{NCT04188185, Co..."
V03AB,ETHANOL,"[{6, Rapid reacqu...","[{null, null, nul..."
A04AD,DIPHENHYDRAMINE,"[{3, Diphenhydram...","[{NCT01967433, Us..."
A03BA,ATROPINE,"[{null, null, nul...","[{null, null, nul..."
6302001,ISOPRENALINE,"[{null, null, nul...","[{null, null, nul..."
R01AD,BETAMETHASONE,"[{null, null, nul...","[{NCT04153396, Pr..."
HELLO,CORONA,"[{null, null, nul...","[{null, null, nul..."


In [13]:
# Journals with one final agg
journals: DataFrame = (
    drugs.join(
        prepared_pubmed_final,
        f.array_contains(f.col("pubmed_title_array"), f.upper(f.col("drug"))),
        "outer",
    )
    .join(
        prepared_clinical_trials_final,
        f.array_contains(
            f.col("clinical_trials_scientific_title_array"), f.upper(f.col("drug"))
        ),
        "left",
    )
    .groupby("atccode", "drug")
    .agg(
        f.collect_set(
            f.struct(
                f.col("pubmed_id"),
                f.col("pubmed_title"),
                f.col("pubmed_date"),
                f.col("pubmed_journal"),
            )
        ).alias("pubmeds"),
        f.collect_set(
            f.struct(
                f.col("clinical_trials_id"),
                f.col("clinical_trials_scientific_title"),
                f.col("clinical_trials_date"),
                f.col("clinical_trials_journal"),
            )
        ).alias("clinical_trials"),
        f.collect_set(
            f.struct(
                f.col("pubmed_date").alias("date"),
                f.col("pubmed_journal").alias("journal"),
            )
        ).alias("journals_pubmed"),
        f.collect_set(
            f.struct(
                f.col("clinical_trials_date").alias("date"),
                f.col("clinical_trials_journal").alias("journal"),
            )
        ).alias("journals_clinical_trials"),
    )
    .withColumn(
        "journals", f.array_union("journals_pubmed", "journals_clinical_trials")
    )
    .drop("journals_pubmed", "journals_clinical_trials")
)

journals

write_json(journals, "journals.json", "journals_single.json")